<a href="https://colab.research.google.com/github/rahuljungbahadur/hands_on_ml_book/blob/main/chp13_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [6]:
base_path='E:\\Projects\\aclImdb'

In [9]:
# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

Saving aclImdb_v1.tar.gz to aclImdb_v1.tar.gz


KeyboardInterrupt: ignored

In [8]:
os.listdir(base_path)

['imdb.vocab', 'imdbEr.txt', 'README', 'test', 'train']

In [ ]:
import tarfile

# open file
file = tarfile.open('aclImdb_v1.tar.gz')
  
# print file names
print(file.getnames())
  
# extract files
file.extractall('./imdb_dataset')
  
# close file
file.close()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
os.listdir('imdb_dataset/aclImdb/train/')

['neg',
 'urls_pos.txt',
 'urls_neg.txt',
 'labeledBow.feat',
 'unsupBow.feat',
 'urls_unsup.txt',
 'unsup',
 'pos']

In [9]:
trainSet_posFp = f'{base_path}/train/pos'
trainSet_negFp = f'{base_path}/train/neg'

In [5]:
import tensorflow as tf

In [ ]:
help(tf.data.Dataset.list_files)

Help on function list_files in module tensorflow.python.data.ops.dataset_ops:

list_files(file_pattern, shuffle=None, seed=None, name=None)
    A dataset of all files matching one or more glob patterns.
    
    The `file_pattern` argument should be a small number of glob patterns.
    If your filenames have already been globbed, use
    `Dataset.from_tensor_slices(filenames)` instead, as re-globbing every
    filename with `list_files` may result in poor performance with remote
    storage systems.
    
    Note: The default behavior of this method is to return filenames in
    a non-deterministic random shuffled order. Pass a `seed` or `shuffle=False`
    to get results in a deterministic order.
    
    Example:
      If we had the following files on our filesystem:
    
        - /path/to/dir/a.txt
        - /path/to/dir/b.py
        - /path/to/dir/c.py
    
      If we pass "/path/to/dir/*.py" as the directory, the dataset
      would produce:
    
        - /path/to/dir/b.py
    

In [12]:
trainSet_posFp_random = tf.data.Dataset.list_files(file_pattern=f'{trainSet_posFp}\\*.txt')
trainSet_negFp_random = tf.data.Dataset.list_files(file_pattern=f'{trainSet_negFp}\\*.txt')

In [ ]:
os.listdir('imdb_dataset/aclImdb')

['README', 'imdbEr.txt', 'valSet', 'testSet', 'test', 'imdb.vocab', 'train']

In [16]:
os.mkdir(f'{base_path}\\testSet')
os.mkdir(f'{base_path}\\valSet')

In [18]:
os.mkdir(f'{base_path}\\testSet\\pos')
os.mkdir(f'{base_path}\\testSet\\neg')
os.mkdir(f'{base_path}\\valSet\\pos')
os.mkdir(f'{base_path}\\valSet\\neg')

### Split the test set into a validation set (15,000) and a test set (10,000).

In [19]:
os.listdir(base_path)

['imdb.vocab', 'imdbEr.txt', 'README', 'test', 'testSet', 'train', 'valSet']

In [20]:
import shutil

## test set
[shutil.copy2(f'{base_path}\\test\\neg\\{fp}', f'{base_path}\\testSet\\neg') for fp in os.listdir(f'{base_path}\\test\\neg')[:5000]]
[shutil.copy2(f'{base_path}\\test\\pos\\{fp}', f'{base_path}\\testSet\\pos') for fp in os.listdir(f'{base_path}\\test\\pos')[:5000]]

## Valid Set
[shutil.copy2(f'{base_path}\\test\\neg\\{fp}', f'{base_path}\\valSet\\neg') for fp in os.listdir(f'{base_path}\\test\\neg')[5000:]]
[shutil.copy2(f'{base_path}\\test\\pos\\{fp}', f'{base_path}\\valSet\\pos') for fp in os.listdir(f'{base_path}\\test\\pos')[5000:]]



['E:\\Projects\\aclImdb\\valSet\\pos\\3250_10.txt',
 'E:\\Projects\\aclImdb\\valSet\\pos\\3251_8.txt',
 'E:\\Projects\\aclImdb\\valSet\\pos\\3252_9.txt',
 'E:\\Projects\\aclImdb\\valSet\\pos\\3253_9.txt',
 'E:\\Projects\\aclImdb\\valSet\\pos\\3254_8.txt',
 'E:\\Projects\\aclImdb\\valSet\\pos\\3255_9.txt',
 'E:\\Projects\\aclImdb\\valSet\\pos\\3256_10.txt',
 'E:\\Projects\\aclImdb\\valSet\\pos\\3257_8.txt',
 'E:\\Projects\\aclImdb\\valSet\\pos\\3258_7.txt',
 'E:\\Projects\\aclImdb\\valSet\\pos\\3259_7.txt',
 'E:\\Projects\\aclImdb\\valSet\\pos\\325_10.txt',
 'E:\\Projects\\aclImdb\\valSet\\pos\\3260_10.txt',
 'E:\\Projects\\aclImdb\\valSet\\pos\\3261_7.txt',
 'E:\\Projects\\aclImdb\\valSet\\pos\\3262_7.txt',
 'E:\\Projects\\aclImdb\\valSet\\pos\\3263_10.txt',
 'E:\\Projects\\aclImdb\\valSet\\pos\\3264_8.txt',
 'E:\\Projects\\aclImdb\\valSet\\pos\\3265_7.txt',
 'E:\\Projects\\aclImdb\\valSet\\pos\\3266_10.txt',
 'E:\\Projects\\aclImdb\\valSet\\pos\\3267_8.txt',
 'E:\\Projects\\aclImdb\\v

In [21]:
(len(os.listdir(f'{base_path}\\valSet\\pos\\')), len(os.listdir(f'{base_path}\\valSet\\neg\\')))

(7500, 7500)

In [22]:
(len(os.listdir(f'{base_path}\\testSet\\pos\\')), len(os.listdir(f'{base_path}\\testSet\\neg\\')))

(5000, 5000)

#### Qns 2 Done

## Use tf.data to create an efficient dataset for each set.

In [ ]:
help(tf.data.TextLineDataset)

In [23]:
import tensorflow as tf
import os

In [75]:
base_path

'E:\\Projects\\aclImdb'

In [74]:
os.listdir('../aclImdb/train')

['labeledBow.feat',
 'neg',
 'pos',
 'unsup',
 'unsupBow.feat',
 'urls_neg.txt',
 'urls_pos.txt',
 'urls_unsup.txt']

In [77]:
trainSet_posFp_random = tf.data.Dataset.list_files(file_pattern=f'{base_path}\\train\\pos\\*.txt')
trainSet_negFp_random = tf.data.Dataset.list_files(file_pattern=f'{base_path}\\train\\neg\\*.txt')

In [79]:
[i for i in trainSet_posFp_random]

[<tf.Tensor: shape=(), dtype=string, numpy=b'E:\\Projects\\aclImdb\\train\\pos\\9367_9.txt'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'E:\\Projects\\aclImdb\\train\\pos\\2363_7.txt'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'E:\\Projects\\aclImdb\\train\\pos\\11856_10.txt'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'E:\\Projects\\aclImdb\\train\\pos\\8844_10.txt'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'E:\\Projects\\aclImdb\\train\\pos\\11166_7.txt'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'E:\\Projects\\aclImdb\\train\\pos\\5347_7.txt'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'E:\\Projects\\aclImdb\\train\\pos\\2354_10.txt'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'E:\\Projects\\aclImdb\\train\\pos\\1380_7.txt'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'E:\\Projects\\aclImdb\\train\\pos\\6168_10.txt'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'E:\\Projects\\aclImdb\\train\\pos\\10519_9.txt'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'E:\\Proj

In [3]:
# trainSet_posFp_random = tf.data.Dataset.list_files(file_pattern='imdb_dataset/aclImdb/train/pos/*.txt')
# trainSet_negFp_random = tf.data.Dataset.list_files(file_pattern='imdb_dataset/aclImdb/train/neg/*.txt')

InvalidArgumentError: ignored

In [81]:
[i for i in trainSet_posFp_random.take(2)]

[<tf.Tensor: shape=(), dtype=string, numpy=b'E:\\Projects\\aclImdb\\train\\pos\\10721_9.txt'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'E:\\Projects\\aclImdb\\train\\pos\\7719_9.txt'>]

In [82]:
trainSet_pos = tf.data.TextLineDataset(trainSet_posFp_random, num_parallel_reads=5)
trainSet_pos = trainSet_pos.map(lambda content: (content, 1))

trainSet_neg = tf.data.TextLineDataset(trainSet_negFp_random, num_parallel_reads=5)
trainSet_neg = trainSet_neg.map(lambda content: (content, 0))

In [83]:
trainSet_complete = tf.data.Dataset.concatenate(trainSet_pos, trainSet_neg).shuffle(25000).batch(32).prefetch(1)

In [84]:
[i for i in trainSet_complete.take(3)]

[(<tf.Tensor: shape=(32,), dtype=string, numpy=
  array([b"Absolutely one of the worst movies I've seen in a long time! It starts off badly and just deteriorates. Katherine Heigl is woefully miscast in a Lolita role and Leo Grillo manfully struggles with what is essentially a cardboard cutout character. The only cast-member with any enthusiasm is Tom Sizemore, who hams it up as a villain and goes completely overboard with his role. The script is dire, the acting horrible and it has plot holes big enough to drive a double-decker bus through! It is also the most sexist movie I have ever seen! Katherine Heigl's character is completely unsympathetic. She's seen as an evil, wanton seductress who lures the poor, innocent married man to cheat on his wife. It is implied throughout the movie that she's underage, and the message that accompanies that plot-strand just beggars belief! At the end, she isn't even able to redeem herself by shooting the man who's obviously (ha!) become demented with r

In [85]:
testSet_pos = tf.data.TextLineDataset(tf.data.Dataset.list_files(f'{base_path}\\testSet\\pos\\'), num_parallel_reads=5).batch(1)
testSet_pos = testSet_pos.map(lambda content: (content, 1))

testSet_neg = tf.data.TextLineDataset(tf.data.Dataset.list_files(f'{base_path}\\testSet\\neg\\'), num_parallel_reads=5).batch(1)
testSet_neg = testSet_neg.map(lambda content: (content,0))

testSet_complete = tf.data.Dataset.concatenate(testSet_pos, testSet_neg).shuffle(10000).batch(32).prefetch(1)

In [86]:
testSet_complete.take(4)

<TakeDataset shapes: ((None, None), (None,)), types: (tf.string, tf.int32)>

In [87]:
valSet_pos = tf.data.TextLineDataset(tf.data.Dataset.list_files(f'{base_path}\\valSet\\pos\\'), num_parallel_reads=5).batch(1)
valSet_pos = valSet_pos.map(lambda content: (content,1))


valSet_neg = tf.data.TextLineDataset(tf.data.Dataset.list_files(f'{base_path}\\valSet\\neg\\'), num_parallel_reads=5).batch(1)
valSet_neg = valSet_neg.map(lambda content: (content, 0))

valSet_complete = tf.data.Dataset.concatenate(valSet_pos, valSet_neg).shuffle(15000).batch(32).prefetch(1)

## Qns 4 Create a binary classification model, using a TextVectorization layer

In [88]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TextVectorization, Dense, Input, Embedding

In [89]:
help(TextVectorization)

Help on class TextVectorization in module keras.layers.preprocessing.text_vectorization:

class TextVectorization(keras.engine.base_preprocessing_layer.PreprocessingLayer)
 |  TextVectorization(max_tokens=None, standardize='lower_and_strip_punctuation', split='whitespace', ngrams=None, output_mode='int', output_sequence_length=None, pad_to_max_tokens=False, vocabulary=None, idf_weights=None, sparse=False, ragged=False, **kwargs)
 |  
 |  A preprocessing layer which maps text features to integer sequences.
 |  
 |  This layer has basic options for managing text in a Keras model. It transforms
 |  a batch of strings (one example = one string) into either a list of token
 |  indices (one example = 1D tensor of integer token indices) or a dense
 |  representation (one example = 1D tensor of float values representing data
 |  about the example's tokens).
 |  
 |  The vocabulary for the layer must be either supplied on construction or
 |  learned via `adapt()`. When this layer is adapted, it

In [90]:
model1 = Sequential()

# model1.add(Input(shape=(1,), dtype=tf.string))




In [97]:
tf_layer = TextVectorization(output_mode='int')

In [98]:
tf_layer.adapt([i for i in trainSet_pos.take(500).as_numpy_iterator()])

RuntimeError: in user code:

    File "C:\Users\rahul\anaconda3\envs\tensorflow_env\lib\site-packages\keras\engine\base_preprocessing_layer.py", line 117, in adapt_step  *
        self._adapt_maybe_build(data)
    File "C:\Users\rahul\anaconda3\envs\tensorflow_env\lib\site-packages\keras\engine\base_preprocessing_layer.py", line 280, in _adapt_maybe_build  **
        self.build(data_shape)
    File "C:\Users\rahul\anaconda3\envs\tensorflow_env\lib\site-packages\keras\layers\preprocessing\text_vectorization.py", line 446, in build
        raise RuntimeError(

    RuntimeError: When using TextVectorization to tokenize strings, the innermost dimension of the input array must be 1, got shape (None, 2)


In [30]:
model1.add(tf_layer)

In [31]:
sample_file = []
with open(f'{trainSet_posFp}/7493_7.txt') as file:
  sample_file=file.read()

In [32]:
sample_file.count('')

2831

In [33]:
model1.predict([sample_file]).size

515

## Creating complete train dataset

In [35]:
trainSet = trainSet_pos.concatenate(trainSet_neg)

In [36]:
y_pos = tf.repeat(1, repeats=12500)
y_neg = tf.repeat(0, repeats=12500)
y_train = tf.concat([y_pos, y_neg], axis=0)

In [51]:
trainSet_complete = tf.data.Dataset.concatenate(trainSet, y_train)

TypeError: Invalid `dataset_or_iterator`. `dataset_or_iterator` must be a `tf.data.Dataset` or tf.data.Iterator object, but got <class 'tensorflow.python.framework.ops.EagerTensor'>.

In [37]:
y_train

<tf.Tensor: shape=(25000,), dtype=int32, numpy=array([1, 1, 1, ..., 0, 0, 0])>

In [40]:
testSet = testSet_pos.concatenate(testSet_neg)
y_test_pos = tf.repeat(1, repeats=5000)
y_test_neg = tf.repeat(0, repeats=5000)
y_test = tf.concat([y_test_pos, y_test_neg], axis=0)

In [41]:
valSet = valSet_pos.concatenate(valSet_neg)
y_val_pos = tf.repeat(1, repeats=7500)
y_val_neg = tf.repeat(0, repeats=7500)
y_val = tf.concat([y_val_pos, y_val_neg], axis=0)

In [43]:
help(Embedding)

Help on class Embedding in module keras.layers.embeddings:

class Embedding(keras.engine.base_layer.Layer)
 |  Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None, **kwargs)
 |  
 |  Turns positive integers (indexes) into dense vectors of fixed size.
 |  
 |  e.g. `[[4], [20]] -> [[0.25, 0.1], [0.6, -0.2]]`
 |  
 |  This layer can only be used as the first layer in a model.
 |  
 |  Example:
 |  
 |  >>> model = tf.keras.Sequential()
 |  >>> model.add(tf.keras.layers.Embedding(1000, 64, input_length=10))
 |  >>> # The model will take as input an integer matrix of size (batch,
 |  >>> # input_length), and the largest integer (i.e. word index) in the input
 |  >>> # should be no larger than 999 (vocabulary size).
 |  >>> # Now model.output_shape is (None, 10, 64), where `None` is the batch
 |  >>> # dimension.
 |  >>> input_array = np.random.randint(1000, 

In [47]:
tf_layer

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc3 in position 4: unexpected end of data

In [48]:
model1.add(Embedding(input_dim=1050, output_dim=64))

In [49]:
model1.compile(optimizer='nadam', loss='binary_crossentropy')

In [50]:
model1.fit(trainSet, y=y_train, epochs=5, validation_data=(valSet, y_val))

ValueError: `y` argument is not supported when using dataset as input.